# **TRABAJO PRÁCTICO: SPARK**
- Alumno: Batallan David Leonardo
- Padrón: 97529
- mail: dbatallan@fi.uba.ar

## **[Notebook](https://colab.research.google.com/drive/1rABNJJeE9ZSiq8cGi-V2a80kiBsAVYug?usp=sharing)**

# **ENUNCIADO**
Trabajaremos resolviendo las consultas de la primera entrega, pero esta vez utilizando Spark. Se debe utilizar la api de RDD (la vista en clase) y NO otra, ya sea la de SQL, Dataframe, etc.

Estas consultas deben estar resueltas, pero no son la totalidad de la entrega. Debe tomar lo aprendido sobre los datos en el TP de Pandas e intentar descubrir más información sobre el set de datos.

- ¿Cuál es la proporción de transacciones fraudulentas?
- ¿Qué relación existe entre el tipo de producto y el monto de la transacción y la columna target? Calcular qué proporción de transacciones son fraudulentas para cada tipo de producto, y verificar si existe impacto.
- ¿Cuál es el impacto del tiempo en la probabilidad de fraude?¿Cómo se distribuyen los casos fraudulentos a lo largo del tiempo?
- ¿Hay alguna plataforma más utilizada que otras para cometer fraude?
- ¿Hay alguna tarjeta preferida por los fraudulentos?
- En los datos vemos gran cantidad de valores nulos, ¿cómo los manejarías?

Importante: Estas preguntas deben usarse como puntapié para desarrollar el análisis, intentando descubrir la relación que existe entre los distintos atributos y el target.

# **Instalamos e importamos librerías**

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=14ef8c7b25b5221efc02349944ab2bdee8c4c7ef620b36151e152410d2c1eb34
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]


In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [3]:
from datetime import datetime


# **Autenticamos con Google Drive**

In [4]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# **Bajamos archivo con los datos transacciones_train.csv**

In [5]:
# transacciones_train.csv en https://drive.google.com/file/d/127v-fngGHW5IkzoYg7TL7KhVyZrTR3hN/view?usp=drive_link

downloaded = drive.CreateFile({'id':"127v-fngGHW5IkzoYg7TL7KhVyZrTR3hN"})
downloaded.GetContentFile('transacciones_train.csv')

## **Creamos el Spark Context**

In [6]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

## **Leemos CSV transacciones_train.csv**

In [7]:
sqlContext = SQLContext(sc)
df = sqlContext.read.csv('transacciones_train.csv', header=True, inferSchema=True)
rdd = df.rdd

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


## **Consultas**

In [8]:
rdd.count()

501959

In [9]:
rdd.first()

Row(_c0=0, Id=2987000, target=0, timestamp=86400, ammount=68.5, product_category='W', feat_card_1=13926, feat_card_2=None, feat_card_3=150.0, feat_card_4='discover', feat_card_5=142.0, feat_card_6='credit', addr1=315.0, addr2=87.0, dist1=19.0, dist2=None, payer_email_domain=None, receiver_email_domain=None, A1=1.0, A2=1.0, A3=0.0, A4=0.0, A5=0.0, A6=1.0, A7=0.0, A8=0.0, A9=1.0, A10=0.0, A11=2.0, A12=0.0, A13=1.0, A14=1.0, B1=14.0, B2=None, B3=13.0, B4=None, B5=None, B6=None, B7=None, B8=None, B9=None, B10=13.0, B11=13.0, B12=None, B13=None, B14=None, B15=0.0, C1='T', C2='T', C3='T', C4='M2', C5='F', C6='T', C7=None, C8=None, C9=None, D1=1.0, D2=1.0, D3=1.0, D4=1.0, D5=1.0, D6=1.0, D7=1.0, D8=1.0, D9=1.0, D10=0.0, D11=0.0, D12=1.0, D13=1.0, D14=1.0, D15=0.0, D16=0.0, D17=0.0, D18=0.0, D19=1.0, D20=1.0, D21=0.0, D22=0.0, D23=1.0, D24=1.0, D25=1.0, D26=1.0, D27=0.0, D28=0.0, D29=0.0, D30=0.0, D31=0.0, D32=0.0, D33=0.0, D34=0.0, D35=None, D36=None, D37=None, D38=None, D39=None, D40=None, D

In [10]:
df = sqlContext.read.csv('transacciones_train.csv', header=True, inferSchema=True)

df_filtrado = df.select(
    'Id', 'target', 'timestamp', 'ammount', 'product_category',
    'feat_card_1', 'feat_card_2', 'feat_card_3', 'feat_card_4',
    'feat_card_5', 'feat_card_6', 'addr1', 'addr2', 'dist1',
    'dist2', 'payer_email_domain', 'receiver_email_domain'
)

rdd_transacciones = df_filtrado.rdd

rdd_transacciones.first()

Row(Id=2987000, target=0, timestamp=86400, ammount=68.5, product_category='W', feat_card_1=13926, feat_card_2=None, feat_card_3=150.0, feat_card_4='discover', feat_card_5=142.0, feat_card_6='credit', addr1=315.0, addr2=87.0, dist1=19.0, dist2=None, payer_email_domain=None, receiver_email_domain=None)

In [11]:
rdd_transacciones.count()

501959

# **Bajamos archivo con los datos identity_features_train.csv**

In [12]:
# transacciones_train.csv en https://drive.google.com/file/d/1u0sbL4T5wy6eag2aZclxexOxq9C3VblZ/view?usp=drive_link

downloaded = drive.CreateFile({'id':"1u0sbL4T5wy6eag2aZclxexOxq9C3VblZ"})
downloaded.GetContentFile('identity_features.csv')

In [13]:
df = sqlContext.read.csv('identity_features.csv', header=True, inferSchema=True)
rdd = df.rdd

In [14]:
rdd.first()

Row(_c0=0, Id=2987004, feature01=0.0, feature02=70787.0, feature03=None, feature04=None, feature05=None, feature06=None, feature07=None, feature08=None, feature09=None, feature10=None, feature11=100.0, feature12='NotFound', feature13=None, feature14=-480.0, feature15='New', feature16='NotFound', feature17=166.0, feature18=None, feature19=542.0, feature20=144.0, feature21=None, feature22=None, feature23=None, feature24=None, feature25=None, feature26=None, feature27=None, feature28='New', feature29='NotFound', feature30='Android 7.0', feature31='samsung browser 6.2', feature32=32.0, feature33='2220x1080', feature34='match_status:2', feature35='T', feature36='F', feature37='T', feature38='T', Device='mobile', DeviceMetadata='SAMSUNG SM-G892A Build/NRD90M')

In [15]:
df = sqlContext.read.csv('identity_features.csv', header=True, inferSchema=True)

df_filtrado = df.select(
    'Id', 'feature30', 'feature31', 'Device', 'DeviceMetadata')

rdd_dispositivo = df_filtrado.rdd

rdd_dispositivo.first()

Row(Id=2987004, feature30='Android 7.0', feature31='samsung browser 6.2', Device='mobile', DeviceMetadata='SAMSUNG SM-G892A Build/NRD90M')

In [16]:
rdd_dispositivo.count()

125565

# **Preguntas**

## **¿Cuál es la proporción de transacciones fraudulentas?**

Analizo cantidad de valores posible del registro target

In [17]:
rdd_transacciones.map(lambda x: x.target).distinct().count()

2

In [18]:
rdd_transacciones.map(lambda x: x.target).distinct().collect()

[0, 1]

Cuento la cantidad de operaciones legitimas y fradulentas

In [19]:
rdd_transacciones.map(lambda x: (x.target, 1)).reduceByKey(lambda x, y: x+y).collect()

[(0, 484379), (1, 17580)]

In [20]:
rdd_transacciones.map(lambda x: ((x.target,x.product_category), 1)).reduceByKey(lambda x, y: x+y).map(lambda x: (x[0],x[1]/501959)).takeOrdered(10, lambda x: -x[1])

[((0, 'W'), 0.722222332899699),
 ((0, 'C'), 0.10437705071529746),
 ((0, 'R'), 0.06433792401371427),
 ((0, 'H'), 0.05787723698549085),
 ((0, 'S'), 0.01616267464075751),
 ((1, 'W'), 0.015292085608585563),
 ((1, 'C'), 0.013423407091017393),
 ((1, 'H'), 0.0028089943600971396),
 ((1, 'R'), 0.0024185242220978206),
 ((1, 'S'), 0.0010797694632430138)]

In [21]:
484379/501959

0.964977219254959

In [22]:
cuenta = rdd_transacciones.map(lambda x: (x.target, 1)).reduceByKey(lambda x, y: x+y).collect()
cuenta

[(0, 484379), (1, 17580)]

In [23]:
100*cuenta[0][1]/(cuenta[0][1]+cuenta[1][1])

96.49772192549591

In [24]:
100*cuenta[1][1]/(cuenta[0][1]+cuenta[1][1])

3.502278074504093

Cuento la cantidad de registros que tengo. hago un map segun target y un reduceByKey para sumar las operaciones legitimas y las fraudulentas por separadas.

In [25]:
total_operaciones = rdd_transacciones.count()
rdd_transacciones.map(lambda x: (x.target, 100/total_operaciones)).reduceByKey(lambda x, y: x+y).collect()

[(0, 96.49772192551232), (1, 3.5022780745041846)]

In [26]:
total_operaciones = rdd_transacciones.count()
total_operaciones

501959

In [27]:
rdd_transacciones.map(lambda x: (x.target, 1)).reduceByKey(lambda x, y: x+y).map(lambda x: (x[0], 100*x[1]/total_operaciones)).collect()

[(0, 96.49772192549591), (1, 3.502278074504093)]

In [28]:
rdd_transacciones.map(lambda x: (x.target, 1)) \
    .reduceByKey(lambda x, y: x+y) \
    .map(lambda x: (x[0], 100*x[1]/total_operaciones)).collect()

[(0, 96.49772192549591), (1, 3.502278074504093)]

## **¿Qué relación existe entre el tipo de producto y el monto de la transacción y la columna target? Calcular qué proporción de transacciones son fraudulentas para cada tipo de producto, y verificar si existe impacto.**

### **Analizo relacion entre el tipo de producto y el target**

Analizo el formato del map para product category y el target

In [29]:
rdd_transacciones.map(lambda x: (x.product_category, (1, x.target))).collect()

[('W', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('H', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('H', (1, 0)),
 ('W', (1, 0)),
 ('C', (1, 0)),
 ('C', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('H', (1, 0)),
 ('H', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('H', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('S', (1, 0)),
 ('W', (1, 0)),
 ('C', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('C', (1, 0)),
 ('C', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('H', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('W', (1, 0)),
 ('W', (

Calculo la cantidad de transacciones segun el tipo de producto:
 el formato es:

  **[('Tipo de producto'),(Cantidad de ventas totales,cantidad de ventas fraudulentas)]**

In [30]:
rdd_transacciones.map(lambda x: (x.product_category, (1, x.target))) \
    .reduceByKey(lambda x,y: (x[0]+y[0], (x[1]+y[1]))).collect()

[('H', (30462, 1410)),
 ('S', (8655, 542)),
 ('R', (33509, 1214)),
 ('C', (59131, 6738)),
 ('W', (370202, 7676))]

Calculo la cantidad de transacciones segun el tipo de producto:
 el formato es:

  **[('Tipo de producto'),(cantidad de ventas legitimas,cantidad de ventas fraudulentas)]**

In [31]:
rdd_transacciones.map(lambda x: (x.product_category, (1, x.target))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
    .map(lambda x: (x[0], x[1][0] - x[1][1], x[1][1])) \
    .collect()

[('H', 29052, 1410),
 ('S', 8113, 542),
 ('R', 32295, 1214),
 ('C', 52393, 6738),
 ('W', 362526, 7676)]

Calculo la proporcion de transacciones segun el tipo de producto:
 el formato es:

  **[('Tipo de producto'),(proporcion de ventas legitimas,proporcion de ventas fraudulentas)]**

In [32]:
rdd_transacciones.map(lambda x: (x.product_category, (1, x.target))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
    .map(lambda x: (x[0], 100*(x[1][0] - x[1][1])/x[1][0], 100* x[1][1]/x[1][0])) \
    .collect()

[('H', 95.3712822532992, 4.628717746700808),
 ('S', 93.73772385904101, 6.262276140958983),
 ('R', 96.37709272135844, 3.622907278641559),
 ('C', 88.60496186433512, 11.39503813566488),
 ('W', 97.92653740390381, 2.0734625960961854)]

In [33]:
rdd_transacciones.map(lambda x: (x.product_category, (1, x.target))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
    .map(lambda x: (x[0], 100*(x[1][0] - x[1][1])/x[1][0], 100*x[1][1]/x[1][0])) \
    .takeOrdered(10, lambda x: -x[2])

[('C', 88.60496186433512, 11.39503813566488),
 ('S', 93.73772385904101, 6.262276140958983),
 ('H', 95.3712822532992, 4.628717746700808),
 ('R', 96.37709272135844, 3.622907278641559),
 ('W', 97.92653740390381, 2.0734625960961854)]

### **Analizo relacion entre el monto y el target**

Analizo el formato del map para monto y el target

In [34]:
rdd_transacciones.map(lambda x: (x.ammount, (1, x.target))) \
    .reduceByKey(lambda x,y: (x[0]+y[0], (x[1]+y[1]))).collect()

[(50.0, (15484, 304)),
 (15.0, (2602, 130)),
 (40.0, (2685, 46)),
 (57.95, (19709, 74)),
 (30.0, (3185, 54)),
 (100.0, (18296, 456)),
 (47.95, (8093, 27)),
 (107.95, (20073, 150)),
 (20.0, (2393, 59)),
 (35.0, (979, 17)),
 (200.0, (5856, 385)),
 (25.0, (6821, 100)),
 (39.5, (223, 9)),
 (3.595, (1, 0)),
 (130.95, (194, 2)),
 (210.95, (647, 4)),
 (500.0, (1272, 117)),
 (80.0, (575, 35)),
 (250.0, (3051, 187)),
 (300.0, (2204, 310)),
 (445.0, (2160, 205)),
 (6.767, (21, 1)),
 (280.0, (2463, 123)),
 (60.0, (926, 41)),
 (44.5, (1236, 19)),
 (140.0, (60, 4)),
 (75.0, (4503, 95)),
 (42.95, (1438, 2)),
 (150.0, (7406, 513)),
 (1153.95, (1, 0)),
 (87.95, (2042, 10)),
 (3000.0, (3, 0)),
 (129.299, (10, 0)),
 (92.95, (210, 2)),
 (973.95, (5, 0)),
 (32.325, (31, 3)),
 (19.455, (62, 6)),
 (67.95, (4504, 15)),
 (37.098, (8, 3)),
 (24.95, (149, 0)),
 (54.5, (2404, 34)),
 (2038.95, (1, 0)),
 (34.5, (953, 11)),
 (450.95, (3, 0)),
 (105.0, (723, 4)),
 (19.576, (72, 2)),
 (335.0, (4689, 215)),
 (82.95, (

Tengo que agrupar los montos segun rangos para poder tener una representacion

In [35]:
bins = [0, 49, 100, 149, 200, 249, float('inf')]
labels = ['0-49', '50-99', '100-149', '150-199', '200-249', '>250']

def assign_range(ammount):
    if(ammount < 50):
        return labels[0]
    elif(ammount < 100):
        return labels[1]
    elif(ammount < 150):
        return labels[2]
    elif(ammount < 200):
        return labels[3]
    elif(ammount < 250):
        return labels[4]
    else:
        return labels[5]

In [36]:
rdd_transacciones.count()

501959

Ahora le aplico el rango al registro ammount y obtengo la cantidad de operaciones legitimas y fraudulentas seun el rango de precios

In [37]:
rdd_transacciones.map(lambda x: (assign_range(x.ammount), (1, x.target))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
    .map(lambda x: (x[0], x[1][0] - x[1][1], x[1][1])) \
    .collect()

[('200-249', 27925, 1072),
 ('50-99', 132841, 3958),
 ('0-49', 152913, 6272),
 ('100-149', 89558, 2175),
 ('>250', 53938, 2884),
 ('150-199', 27204, 1219)]

Ahora le aplico el rango al registro ammount y obtengo la proporcion de operaciones legitimas y fraudulentas seun el rango de precios

In [38]:
rdd_transacciones.map(lambda x: (assign_range(x.ammount), (1, x.target))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
    .map(lambda x: (x[0], 100*(x[1][0] - x[1][1])/x[1][0], 100*x[1][1]/x[1][0])) \
    .collect()

[('200-249', 96.30306583439666, 3.6969341656033383),
 ('50-99', 97.10670399637424, 2.8932960036257573),
 ('0-49', 96.05993026981186, 3.9400697301881458),
 ('100-149', 97.62898847742906, 2.3710115225709396),
 ('>250', 94.92450107352786, 5.075498926472141),
 ('150-199', 95.71121978679238, 4.288780213207613)]

In [39]:
rdd_transacciones.map(lambda x: (assign_range(x.ammount), (1, x.target))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
    .map(lambda x: (x[0], 100*(x[1][0] - x[1][1])/x[1][0], 100*x[1][1]/x[1][0])) \
    .takeOrdered(10, lambda x: -x[2])

[('>250', 94.92450107352786, 5.075498926472141),
 ('150-199', 95.71121978679238, 4.288780213207613),
 ('0-49', 96.05993026981186, 3.9400697301881458),
 ('200-249', 96.30306583439666, 3.6969341656033383),
 ('50-99', 97.10670399637424, 2.8932960036257573),
 ('100-149', 97.62898847742906, 2.3710115225709396)]

### **Tipo de producto, rango y target**

Ahora aplico a la ctegoria de producto y rango del monto. Obtengo en formato

**(('categoria de producto','rango del monto'),cantidad de operaciones legitimas, cantidad de operaciones fraudulentas)**

In [40]:
rdd_transacciones.map(lambda x: ((x.product_category, assign_range(x.ammount)), (1, x.target))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
    .map(lambda x: (x[0], (x[1][0] - x[1][1]), x[1][1])) \
    .collect()

[(('H', '0-49'), 9142, 100),
 (('S', '0-49'), 5124, 306),
 (('H', '>250'), 524, 308),
 (('S', '100-149'), 777, 50),
 (('C', '>250'), 152, 46),
 (('W', '0-49'), 100046, 1412),
 (('R', '50-99'), 4592, 44),
 (('R', '200-249'), 4373, 298),
 (('R', '100-149'), 11870, 188),
 (('C', '150-199'), 826, 117),
 (('C', '0-49'), 38117, 4446),
 (('H', '100-149'), 5377, 238),
 (('C', '100-149'), 2374, 428),
 (('R', '>250'), 5741, 502),
 (('R', '150-199'), 5235, 174),
 (('S', '200-249'), 120, 40),
 (('R', '0-49'), 484, 8),
 (('S', '150-199'), 115, 24),
 (('W', '50-99'), 104069, 1852),
 (('W', '150-199'), 19447, 534),
 (('C', '50-99'), 10618, 1640),
 (('W', '200-249'), 22507, 599),
 (('H', '150-199'), 1581, 370),
 (('C', '200-249'), 306, 61),
 (('S', '>250'), 224, 20),
 (('H', '50-99'), 11809, 320),
 (('W', '>250'), 47297, 2008),
 (('W', '100-149'), 69160, 1271),
 (('H', '200-249'), 619, 74),
 (('S', '50-99'), 1753, 102)]

In [41]:
rdd_transacciones.map(lambda x: ((x.product_category, assign_range(x.ammount)), (1, x.target))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
    .map(lambda x: (x[1][1])) \
    .sum()

17580



Ahora aplico a la categoria de producto y rango del monto. Obtengo en formato

**(('categoria de producto','rango del monto'),proporcion de operaciones legitimas, proporcion de operaciones fraudulentas)**


In [42]:
rdd_transacciones.map(lambda x: ((x.product_category, assign_range(x.ammount)), (1, x.target))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
    .map(lambda x: (x[0], 100*(x[1][0] - x[1][1])/x[1][0], 100*x[1][1]/x[1][0])) \
    .collect()

[(('H', '0-49'), 98.91798312053668, 1.0820168794633196),
 (('S', '0-49'), 94.3646408839779, 5.6353591160221),
 (('H', '>250'), 62.98076923076923, 37.01923076923077),
 (('S', '100-149'), 93.9540507859734, 6.045949214026602),
 (('C', '>250'), 76.76767676767676, 23.232323232323232),
 (('W', '0-49'), 98.6082911155355, 1.3917088844645076),
 (('R', '50-99'), 99.0509059534081, 0.9490940465918896),
 (('R', '200-249'), 93.6202098051809, 6.379790194819097),
 (('R', '100-149'), 98.44086913252612, 1.5591308674738762),
 (('C', '150-199'), 87.59278897136798, 12.407211028632025),
 (('C', '0-49'), 89.55430773206776, 10.445692267932241),
 (('H', '100-149'), 95.7613535173642, 4.238646482635797),
 (('C', '100-149'), 84.72519628836545, 15.274803711634547),
 (('R', '>250'), 91.95899407336216, 8.041005926637835),
 (('R', '150-199'), 96.78313921242373, 3.2168607875762616),
 (('S', '200-249'), 75.0, 25.0),
 (('R', '0-49'), 98.3739837398374, 1.6260162601626016),
 (('S', '150-199'), 82.73381294964028, 17.266187

Hago un takeorder para obtener las primeras 10 mas fraudulentas

In [43]:
rdd_transacciones.map(lambda x: ((x.product_category, assign_range(x.ammount)), (1, x.target))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
    .map(lambda x: (x[0], 100*(x[1][0] - x[1][1])/x[1][0], 100*x[1][1]/x[1][0])) \
    .takeOrdered(10, lambda x: -x[2])

[(('H', '>250'), 62.98076923076923, 37.01923076923077),
 (('S', '200-249'), 75.0, 25.0),
 (('C', '>250'), 76.76767676767676, 23.232323232323232),
 (('H', '150-199'), 81.03536647872886, 18.964633521271143),
 (('S', '150-199'), 82.73381294964028, 17.26618705035971),
 (('C', '200-249'), 83.37874659400545, 16.62125340599455),
 (('C', '100-149'), 84.72519628836545, 15.274803711634547),
 (('C', '50-99'), 86.62098221569588, 13.379017784304128),
 (('C', '150-199'), 87.59278897136798, 12.407211028632025),
 (('H', '200-249'), 89.32178932178932, 10.678210678210679)]

## **Relacion entre tipo de producto y medio de pago (feat_card_4).**

In [44]:
tot_fraud=rdd_transacciones.filter(lambda x: x.target == 1).count()
tot_fraud

17580

Ahora aplico a la categoria de producto y tipo de tarjeta. Obtengo en formato

**(('categoria de producto','tipo de tarjeta'),proporcion de operaciones legitimas, proporcion de operaciones fraudulentas)**

In [45]:
rdd_transacciones.map(lambda x: ((x.product_category, x.feat_card_4), (1, x.target))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
    .map(lambda x: (x[0], x[1][0] - x[1][1], x[1][1])) \
    .collect()

[(('W', 'mastercard'), 118060, 2135),
 (('W', 'visa'), 240393, 5222),
 (('H', 'mastercard'), 6957, 374),
 (('H', 'visa'), 20220, 914),
 (('C', 'mastercard'), 21100, 2601),
 (('S', 'visa'), 5086, 314),
 (('H', 'american express'), 1334, 82),
 (('R', 'mastercard'), 6352, 323),
 (('C', 'american express'), 0, 2),
 (('R', None), 0, 1),
 (('R', 'american express'), 5426, 100),
 (('S', 'discover'), 473, 53),
 (('R', 'discover'), 940, 42),
 (('H', None), 7, 1),
 (('C', None), 71, 6),
 (('W', 'discover'), 3344, 301),
 (('S', 'american express'), 514, 34),
 (('C', 'visa'), 31222, 4129),
 (('W', None), 729, 18),
 (('R', 'visa'), 19577, 748),
 (('S', 'mastercard'), 2040, 141),
 (('H', 'discover'), 534, 39)]

Ahora aplico a la categoria de producto y rango del monto. Obtengo en formato

**(('categoria de producto','rango del monto'),proporcion de operaciones legitimas, proporcion de operaciones fraudulentas,proporcion de operaciones fraudulentas solo considerando las operaciones fraudulentas)**

In [46]:
transac_prod_tarjeta=rdd_transacciones.map(lambda x: ((x.product_category, x.feat_card_4), (1, x.target))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
    .map(lambda x: (x[0], (x[1][0] - x[1][1])/x[1][0], x[1][1]/x[1][0],x[1][1]/tot_fraud)) \
    .takeOrdered(10, lambda x: -x[3])

In [47]:
transac_prod_tarjeta

[(('W', 'visa'), 0.978739083525029, 0.02126091647497099, 0.2970420932878271),
 (('C', 'visa'), 0.8831999094792227, 0.11680009052077735, 0.23486916951080775),
 (('C', 'mastercard'),
  0.8902577950297456,
  0.10974220497025441,
  0.14795221843003412),
 (('W', 'mastercard'),
  0.9822371978867673,
  0.017762802113232664,
  0.1214448236632537),
 (('H', 'visa'),
  0.9567521529289297,
  0.043247847071070315,
  0.05199089874857793),
 (('R', 'visa'), 0.9631980319803198, 0.0368019680196802, 0.04254835039817975),
 (('H', 'mastercard'),
  0.9489837675624062,
  0.05101623243759378,
  0.021274175199089876),
 (('R', 'mastercard'),
  0.9516104868913857,
  0.04838951310861423,
  0.018373151308304893),
 (('S', 'visa'),
  0.9418518518518518,
  0.05814814814814815,
  0.017861205915813423),
 (('W', 'discover'),
  0.9174211248285322,
  0.08257887517146777,
  0.017121729237770195)]

In [48]:
for prod in transac_prod_tarjeta:
  print(prod)

(('W', 'visa'), 0.978739083525029, 0.02126091647497099, 0.2970420932878271)
(('C', 'visa'), 0.8831999094792227, 0.11680009052077735, 0.23486916951080775)
(('C', 'mastercard'), 0.8902577950297456, 0.10974220497025441, 0.14795221843003412)
(('W', 'mastercard'), 0.9822371978867673, 0.017762802113232664, 0.1214448236632537)
(('H', 'visa'), 0.9567521529289297, 0.043247847071070315, 0.05199089874857793)
(('R', 'visa'), 0.9631980319803198, 0.0368019680196802, 0.04254835039817975)
(('H', 'mastercard'), 0.9489837675624062, 0.05101623243759378, 0.021274175199089876)
(('R', 'mastercard'), 0.9516104868913857, 0.04838951310861423, 0.018373151308304893)
(('S', 'visa'), 0.9418518518518518, 0.05814814814814815, 0.017861205915813423)
(('W', 'discover'), 0.9174211248285322, 0.08257887517146777, 0.017121729237770195)


# **¿Cuál es el impacto del tiempo en la probabilidad de fraude?¿Cómo se distribuyen los casos fraudulentos a lo largo del tiempo?**

Defino funciones para transformar el registro timesamp

In [49]:
def map_timestamp_to_date(timestamp):

    return datetime.fromtimestamp(timestamp+pd.Timestamp('2024-01-01').timestamp()).date()

In [50]:
def map_timestamp_to_year(timestamp):

    return datetime.fromtimestamp(timestamp+pd.Timestamp('2024-01-01').timestamp()).year

In [51]:
def map_timestamp_to_month(timestamp):

    return datetime.fromtimestamp(timestamp+pd.Timestamp('2024-01-01').timestamp()).month

In [52]:
def map_timestamp_to_day(timestamp):

    return datetime.fromtimestamp(timestamp+pd.Timestamp('2024-01-01').timestamp()).day

In [53]:
def map_timestamp_to_hour(timestamp):

    return datetime.fromtimestamp(timestamp+pd.Timestamp('2024-01-01').timestamp()).hour

In [54]:
def map_timestamp_to_ndays(timestamp):

    return int((timestamp/(24*60*60))-1)

In [55]:
def map_nday_to_week_day(timestamp):
    nday=map_timestamp_to_ndays(timestamp)
    return int(((nday-2) % 7)+1)

## **Analizo la distribucion de operaciones segun dia de la semana**

Cuento la cantidad de operaciones legitimas y fraudulentas segun el dia de la semana

In [56]:
rdd_transacciones.map(lambda x: (map_nday_to_week_day(x.timestamp), (1, x.target))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
    .map(lambda x: (x[0], x[1][0] - x[1][1], x[1][1])) \
    .collect()

[(5, 68649, 2706),
 (6, 82668, 3026),
 (1, 57912, 2201),
 (7, 67043, 2563),
 (2, 71624, 2386),
 (3, 68113, 2291),
 (4, 68370, 2407)]

Proporcion de operaciones legitimas y fraudulentas segun el dia de la semana

In [57]:
rdd_transacciones.map(lambda x: (map_nday_to_week_day(x.timestamp), (1, x.target))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
    .map(lambda x: (x[0], (x[1][0] - x[1][1])/x[1][0], x[1][1]/x[1][0])) \
    .collect()

[(5, 0.9620769392474249, 0.03792306075257515),
 (6, 0.9646883095665975, 0.03531169043340257),
 (1, 0.9633856237419527, 0.036614376258047344),
 (7, 0.9631784616268713, 0.03682153837312875),
 (2, 0.967761113363059, 0.03223888663694095),
 (3, 0.9674592352707233, 0.03254076472927674),
 (4, 0.9659917769897001, 0.03400822301029996)]

In [58]:
rdd_transacciones.map(lambda x: (map_nday_to_week_day(x.timestamp), (1, x.target))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
    .map(lambda x: (x[0], 100*(x[1][0] - x[1][1])/x[1][0], 100*x[1][1]/x[1][0])) \
    .takeOrdered(10, lambda x: -x[2])

[(5, 96.20769392474249, 3.7923060752575153),
 (7, 96.31784616268712, 3.6821538373128755),
 (1, 96.33856237419526, 3.6614376258047345),
 (6, 96.46883095665974, 3.5311690433402574),
 (4, 96.59917769897001, 3.400822301029996),
 (3, 96.74592352707232, 3.2540764729276748),
 (2, 96.7761113363059, 3.2238886636940953)]

In [59]:
tot_fraud=rdd_transacciones.filter(lambda x: x.target == 1).count()
tot_fraud

17580

Cuento la cantidad de operaciones legitimas y fraudulentas segun el dia de la semana. Agrego la porporcion de fraudulentas si solo considero las operaciones fraudulentas

In [60]:
rdd_transacciones.map(lambda x: (map_nday_to_week_day(x.timestamp), (1, x.target))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
    .map(lambda x: (x[0], 100*(x[1][0] - x[1][1])/x[1][0], 100*x[1][1]/x[1][0],100* x[1][1]/tot_fraud)) \
    .takeOrdered(10, lambda x: -x[3])

[(6, 96.46883095665974, 3.5311690433402574, 17.212741751990897),
 (5, 96.20769392474249, 3.7923060752575153, 15.392491467576791),
 (7, 96.31784616268712, 3.6821538373128755, 14.579067121729238),
 (4, 96.59917769897001, 3.400822301029996, 13.691695108077361),
 (2, 96.7761113363059, 3.2238886636940953, 13.572241183162685),
 (3, 96.74592352707232, 3.2540764729276748, 13.031854379977247),
 (1, 96.33856237419526, 3.6614376258047345, 12.51990898748578)]

Las operaciones fraudulentas se acumulan sobre los ultimos dias de la semana

## **Analizo la distribucion de operaciones segun la hora**

Hago un map segun la hora del dia y obtengo la cantidad de operaciones legitimas y fraudulentas

In [61]:
rdd_transacciones.map(lambda x: (map_timestamp_to_hour(x.timestamp), (1, x.target))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
    .map(lambda x: (x[0], x[1][0] - x[1][1], x[1][1])) \
    .collect()

[(0, 31423, 1041),
 (5, 8137, 592),
 (10, 2529, 180),
 (15, 27640, 708),
 (20, 34387, 1219),
 (1, 27433, 897),
 (6, 4884, 402),
 (11, 4914, 202),
 (16, 31840, 933),
 (21, 34466, 1199),
 (2, 22278, 848),
 (7, 2941, 356),
 (12, 9266, 291),
 (17, 33639, 1129),
 (22, 33904, 1114),
 (3, 17394, 668),
 (8, 2059, 222),
 (13, 15871, 352),
 (18, 34396, 1257),
 (23, 32847, 1297),
 (4, 12615, 653),
 (9, 1788, 203),
 (14, 22822, 558),
 (19, 34906, 1259)]

Ahora calculo la proporcion de operaciones legitimas y fraudulentas absolutas y la proporciones de fraudulentas si solo consideramos el total de operaciones fraudulentas

In [62]:
rdd_transacciones.map(lambda x: (map_timestamp_to_hour(x.timestamp), (1, x.target))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
    .map(lambda x: (x[0],100* (x[1][0] - x[1][1])/x[1][0],100* x[1][1]/x[1][0],100* x[1][1]/tot_fraud)) \
    .takeOrdered(10, lambda x: -x[3])

[(23, 96.20138238050609, 3.7986176194939083, 7.377701934015927),
 (19, 96.51873358219272, 3.481266417807272, 7.161547212741752),
 (18, 96.47434998457352, 3.525650015426472, 7.150170648464163),
 (20, 96.57641970454418, 3.423580295455822, 6.934015927189988),
 (21, 96.63816066171316, 3.3618393382868357, 6.820250284414107),
 (17, 96.75276115968707, 3.2472388403129315, 6.422070534698521),
 (22, 96.81877891370152, 3.181221086298475, 6.33674630261661),
 (0, 96.79337111877773, 3.206628881222277, 5.921501706484642),
 (16, 97.15314435663504, 2.846855643364965, 5.30716723549488),
 (1, 96.83374514648783, 3.1662548535121777, 5.102389078498294)]

In [63]:
rdd_transacciones.map(lambda x: (map_timestamp_to_hour(x.timestamp), (1, x.target))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
    .map(lambda x: (100* x[1][1]/tot_fraud)) \
    .sum()

99.99999999999999

# **¿Hay alguna tarjeta preferida por los fraudulentos?**

Hago un filter para analizar solo las operaciones fraudulentas.

In [64]:
rdd_fraudulentas = rdd_transacciones.filter(lambda x: x.target == 1)
total_fraudulentas=rdd_fraudulentas.count()

ahora calculo la cantidad de operaciones frauduletnas segun tl tipo de tarjeta

In [65]:
rdd_fraudulentas.map(lambda x: (x.feat_card_4, 1)) \
    .reduceByKey(lambda x, y: x + y) \
    .map(lambda x: (x[0],x[1] )) \
    .collect()

[(None, 26),
 ('visa', 11327),
 ('american express', 218),
 ('mastercard', 5574),
 ('discover', 435)]

calculo la proporcion de operaciones frauduletnas

In [66]:
rdd_fraudulentas = rdd_transacciones.filter(lambda x: x.target == 1)
total_fraudulentas=rdd_fraudulentas.count()

rdd_fraudulentas.map(lambda x: (x.feat_card_4, 1)) \
    .reduceByKey(lambda x, y: x + y) \
    .map(lambda x: (x[0], 100 * x[1] / total_fraudulentas)) \
    .takeOrdered(10, lambda x: -x[1])

[('visa', 64.43117178612059),
 ('mastercard', 31.706484641638227),
 ('discover', 2.474402730375427),
 ('american express', 1.2400455062571103),
 (None, 0.14789533560864618)]

Se puede ver que visa es la tarjeta en la cual se realizan mas operaciones frauduletnas

# **Relacion entre transacciones fraudulentas y payer_email_domain**

Defino una funcion para aquellos registros que no tiene datos y establesca como sin datos

In [67]:
def map_email(email):
  if email is None:
    return 'S/D'
  else:
    return email

Calculo la proporcion de operaciones Legitimas y Fraudulentas para el mail absolutas y proporcion de operaciones frauduletnas relativas

El  formato es:

**('mail', 'Prop Legit Abs', 'Prop Fraud Abs', 'Prop Fraud Rel')**

In [68]:
rdd_transacciones.map(lambda x: (map_email(x.payer_email_domain), (1, x.target))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
    .map(lambda x: (x[0],100* (x[1][0] - x[1][1])/x[1][0],100* x[1][1]/x[1][0],100* x[1][1]/tot_fraud)) \
    .takeOrdered(10, lambda x: -x[3])

[('gmail.com', 95.62195253811419, 4.378047461885809, 48.3674630261661),
 ('S/D', 97.00864509218881, 2.991354907811185, 13.34470989761092),
 ('hotmail.com', 94.83379749099363, 5.166202509006362, 11.501706484641637),
 ('yahoo.com', 97.71486326355166, 2.2851367364483375, 11.188850967007964),
 ('anonymous.com', 97.69800031491104, 2.3019996850889624, 4.158134243458475),
 ('aol.com', 97.73548842813811, 2.2645115718618856, 3.1001137656427757),
 ('outlook.com', 90.94287041337668, 9.057129586623317, 2.218430034129693),
 ('comcast.net', 97.04941860465117, 2.9505813953488373, 1.154721274175199),
 ('icloud.com', 96.76570139150057, 3.234298608499436, 0.9783845278725825),
 ('mail.com', 80.04158004158005, 19.95841995841996, 0.5460750853242321)]

Se puede ver que gmail es la plataforma en la que se realizan mas trnsacciones frauduletntas

# **Relacion entre transacciones fraudulentas y receiver_email_domain**

Hago el mismo analisis para dominos de mail del receptor

In [69]:
rdd_transacciones.map(lambda x: (map_email(x.payer_email_domain), (1, x.target))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
    .map(lambda x: (x[0],100* (x[1][0] - x[1][1])/x[1][0],100* x[1][1]/x[1][0],100* x[1][1]/tot_fraud)) \
    .takeOrdered(10, lambda x: -x[3])

[('gmail.com', 95.62195253811419, 4.378047461885809, 48.3674630261661),
 ('S/D', 97.00864509218881, 2.991354907811185, 13.34470989761092),
 ('hotmail.com', 94.83379749099363, 5.166202509006362, 11.501706484641637),
 ('yahoo.com', 97.71486326355166, 2.2851367364483375, 11.188850967007964),
 ('anonymous.com', 97.69800031491104, 2.3019996850889624, 4.158134243458475),
 ('aol.com', 97.73548842813811, 2.2645115718618856, 3.1001137656427757),
 ('outlook.com', 90.94287041337668, 9.057129586623317, 2.218430034129693),
 ('comcast.net', 97.04941860465117, 2.9505813953488373, 1.154721274175199),
 ('icloud.com', 96.76570139150057, 3.234298608499436, 0.9783845278725825),
 ('mail.com', 80.04158004158005, 19.95841995841996, 0.5460750853242321)]

# **Relacion entre transacciones fraudulentas y addr1**

Defino una funcion para aquellos registros que no tengan datos en addr1

In [70]:
def map_addr(addr):
  if addr is None:
    return 'S/D'
  else:
    return addr

Calculo la proporcion de operaciones Legitimas y Fraudulentas para el registro **addr1** absolutas y proporcion de operaciones frauduletnas relativas

El  formato es:

**('addr1', 'Prop Legit Abs', 'Prop Fraud Abs', 'Prop Fraud Rel')**

In [71]:
rdd_transacciones.map(lambda x: (map_addr(x.addr1), (1, x.target))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
    .map(lambda x: (x[0],100* (x[1][0] - x[1][1])/x[1][0],100* x[1][1]/x[1][0],100* x[1][1]/tot_fraud)) \
    .takeOrdered(10, lambda x: -x[3])

[('S/D', 88.52671133326244, 11.473288666737567, 36.82025028441411),
 (204.0, 97.31147910965667, 2.6885208903433293, 5.420932878270762),
 (325.0, 97.5058628776383, 2.494137122361705, 5.1422070534698525),
 (299.0, 97.79452538186217, 2.2054746181378233, 4.977246871444824),
 (330.0, 96.66430761972163, 3.3356923802783673, 4.021615472127418),
 (264.0, 98.14814814814815, 1.8518518518518519, 3.5665529010238908),
 (272.0, 97.20357288808454, 2.796427111915465, 2.7246871444823664),
 (441.0, 97.47466168393636, 2.525338316063643, 2.5369738339021617),
 (123.0, 97.41322131328764, 2.586778686712359, 1.9965870307167235),
 (126.0, 97.33609703669583, 2.663902963304161, 1.9738339021615472)]

In [72]:
rdd_transacciones.map(lambda x: (map_addr(x.addr1), (1, x.target))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
    .map(lambda x: (x[0],(x[1][0] - x[1][1]),x[1][1])) \
    .takeOrdered(10, lambda x: -x[2])

[('S/D', 49945, 6473),
 (204.0, 34494, 953),
 (325.0, 35341, 904),
 (299.0, 38799, 875),
 (330.0, 20488, 707),
 (264.0, 33231, 627),
 (272.0, 16650, 479),
 (441.0, 17215, 446),
 (123.0, 13218, 351),
 (126.0, 12679, 347)]

# **Promedio de venta segun rango de precio**

Calculo el promedio de precio segun rango de precios para operaciones Legitimas y Frudulentas

In [73]:
rdd_transacciones.map(lambda x: (assign_range(x.ammount), (1, x.target,x.ammount, x.target*x.ammount))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1],x[2] + y[2],x[3] + y[3])) \
    .collect()

[('200-249', (28997, 1072, 6237787.439999818, 228637.09699999995)),
 ('50-99', (136799, 3958, 9016281.97599866, 275251.97900000017)),
 ('0-49', (159185, 6272, 5100452.817999104, 178285.80999999994)),
 ('100-149', (91733, 2175, 10343425.560999775, 252277.32399999976)),
 ('>250', (56822, 2884, 32249044.485001568, 1483058.6940000008)),
 ('150-199', (28423, 1219, 4643202.26899988, 196725.43499999997))]

In [74]:
rdd_transacciones.map(lambda x: (assign_range(x.ammount), (1, x.target,x.ammount, x.target*x.ammount))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1],x[2] + y[2],x[3] + y[3])) \
    .map(lambda x: (x[0],(x[1][0] - x[1][1]),x[1][1],x[1][2]-x[1][3],x[1][3])) \
    .collect()

[('200-249', 27925, 1072, 6009150.342999818, 228637.09699999995),
 ('50-99', 132841, 3958, 8741029.99699866, 275251.97900000017),
 ('0-49', 152913, 6272, 4922167.007999104, 178285.80999999994),
 ('100-149', 89558, 2175, 10091148.236999776, 252277.32399999976),
 ('>250', 53938, 2884, 30765985.791001566, 1483058.6940000008),
 ('150-199', 27204, 1219, 4446476.833999881, 196725.43499999997)]

In [75]:
rdd_transacciones.map(lambda x: (assign_range(x.ammount), (1, x.target,x.ammount, x.target*x.ammount))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1],x[2] + y[2],x[3] + y[3])) \
    .map(lambda x: (x[0],(x[1][0] - x[1][1]),x[1][1],(x[1][2]-x[1][3])/(x[1][0] - x[1][1]),x[1][3]/x[1][1])) \
    .collect()

[('200-249', 27925, 1072, 215.1889111190624, 213.28087406716412),
 ('50-99', 132841, 3958, 65.80069404023351, 69.5431983324912),
 ('0-49', 152913, 6272, 32.18932993270098, 28.425671237244888),
 ('100-149', 89558, 2175, 112.67723974407397, 115.98957425287345),
 ('>250', 53938, 2884, 570.3953760058134, 514.236717753121),
 ('150-199', 27204, 1219, 163.44937634171006, 161.3826374077112)]

Se puede ver que las operaciones fraudulentas tienden a hacerse en valores mas altos que las operaciones legitimas

# **Promedio de venta segun rango de venta y tipo de producto**

Realizo operaciones para el tipo de producto, el rango, cantidad de operaciones legitimas, cantidad de operaciones fraudulentas, monto de operaciones legitimas totales, monto de operaciones fraudulentas totales

In [98]:
rdd_transacciones.map(lambda x: ((x.product_category, assign_range(x.ammount)), (1, x.target, x.ammount, x.target * x.ammount))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1], x[2] + y[2], x[3] + y[3])) \
    .map(lambda x: (x[0], (x[1][0] - x[1][1]), x[1][1], x[1][2] - x[1][3], x[1][3])) \
    .collect()

[(('H', '0-49'), 9142, 100, 252317.0, 2396.0),
 (('S', '0-49'), 5124, 306, 113004.15, 6721.0),
 (('H', '>250'), 524, 308, 171197.0, 107485.0),
 (('S', '100-149'), 777, 50, 80076.5, 5040.0),
 (('C', '>250'), 152, 46, 46306.95899999999, 14754.984),
 (('W', '0-49'), 100046, 1412, 3600585.1499995245, 53877.000000000015),
 (('R', '50-99'), 4592, 44, 268825.0, 2500.0),
 (('R', '200-249'), 4373, 298, 878075.0, 60400.0),
 (('R', '100-149'), 11870, 188, 1225525.0, 19525.0),
 (('C', '150-199'), 826, 117, 140612.48399999994, 19610.125),
 (('C', '0-49'), 38117, 4446, 944160.7080000001, 115091.81000000007),
 (('H', '100-149'), 5377, 238, 543658.0, 24031.0),
 (('C', '100-149'), 2374, 428, 287729.05700000044, 52576.99399999997),
 (('R', '>250'), 5741, 502, 2245625.0, 183625.0),
 (('R', '150-199'), 5235, 174, 793975.0, 27300.0),
 (('S', '200-249'), 120, 40, 24235.5, 8020.0),
 (('R', '0-49'), 484, 8, 12100.0, 200.0),
 (('S', '150-199'), 115, 24, 18316.95, 3890.0),
 (('W', '50-99'), 104069, 1852, 697621

In [ ]:
dominio de mail, fraud, monto

In [99]:
rdd_transacciones.map(lambda x: ((x.payer_email_domain), (1, x.target, x.ammount, x.target * x.ammount))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1], x[2] + y[2], x[3] + y[3])) \
    .map(lambda x: (x[0], (x[1][0] - x[1][1]), x[1][1], x[1][2] - x[1][3], x[1][3])) \
    .collect()

[(None, 76080, 2346, 9082402.793999502, 376426.3809999995),
 ('comcast.net', 6677, 203, 1230595.1850000056, 95897.44999999998),
 ('msn.com', 3445, 62, 609317.9099999996, 7888.842000000001),
 ('q.com', 162, 0, 24033.359999999993, 0.0),
 ('centurylink.net', 174, 0, 39786.04, 0.0),
 ('earthlink.net', 433, 6, 79989.84100000001, 1950.0),
 ('cfl.rr.com', 142, 0, 14866.830000000002, 0.0),
 ('twc.com', 203, 0, 31363.502, 0.0),
 ('yahoo.fr', 101, 5, 4691.857, 250.0),
 ('yahoo.de', 60, 0, 3089.362, 0.0),
 ('gmail.com', 185716, 8503, 23645245.171005223, 1191969.767000001),
 ('verizon.net', 2364, 15, 443055.20999999926, 5924.0),
 ('rocketmail.com', 559, 2, 88120.80000000006, 229.95),
 ('embarqmail.com', 220, 7, 26445.409999999993, 939.99),
 ('bellsouth.net', 1639, 47, 283643.57099999965, 7084.9),
 ('suddenlink.net', 118, 4, 15621.689999999997, 400.0),
 ('windstream.net', 240, 0, 43363.05, 0.0),
 ('outlook.es', 340, 56, 13939.997000000001, 2738.5850000000005),
 ('mac.com', 365, 11, 65653.7900000000

Calculo el promedio

In [77]:
rdd_transacciones.map(lambda x: ((x.product_category, assign_range(x.ammount)), (1, x.target, x.ammount, x.target * x.ammount))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1], x[2] + y[2], x[3] + y[3])) \
    .map(lambda x: (x[0],(x[1][0] - x[1][1]),x[1][1],(x[1][2]-x[1][3])/(x[1][0] - x[1][1]),x[1][3]/x[1][1])) \
    .collect()

[(('H', '0-49'), 9142, 100, 27.599759352439293, 23.96),
 (('S', '0-49'), 5124, 306, 22.05389344262295, 21.964052287581698),
 (('H', '>250'), 524, 308, 326.7118320610687, 348.97727272727275),
 (('S', '100-149'), 777, 50, 103.05855855855856, 100.8),
 (('C', '>250'), 152, 46, 304.6510460526315, 320.76052173913047),
 (('W', '0-49'), 100046, 1412, 35.98929642364037, 38.15651558073655),
 (('R', '50-99'), 4592, 44, 58.54202961672474, 56.81818181818182),
 (('R', '200-249'), 4373, 298, 200.79464898239195, 202.68456375838926),
 (('R', '100-149'), 11870, 188, 103.24557708508846, 103.8563829787234),
 (('C', '150-199'), 826, 117, 170.2330314769975, 167.60790598290598),
 (('C', '0-49'), 38117, 4446, 24.770068683264686, 25.886596941070643),
 (('H', '100-149'), 5377, 238, 101.10805281755626, 100.97058823529412),
 (('C', '100-149'), 2374, 428, 121.20010825610802, 122.84344392523357),
 (('R', '>250'), 5741, 502, 391.15572199965163, 365.78685258964146),
 (('R', '150-199'), 5235, 174, 151.66666666666666, 

In [78]:
monto_tipo_prod = rdd_transacciones.map(lambda x: ((x.product_category, assign_range(x.ammount)), (1, x.target, x.ammount, x.target * x.ammount))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1], x[2] + y[2], x[3] + y[3])) \
    .map(lambda x: (x[0],(x[1][0] - x[1][1]),x[1][1],(x[1][2]-x[1][3])/(x[1][0] - x[1][1]),x[1][3]/x[1][1])) \
    .collect()

In [79]:
for prod in monto_tipo_prod:
  print(prod)

(('H', '0-49'), 9142, 100, 27.599759352439293, 23.96)
(('S', '0-49'), 5124, 306, 22.05389344262295, 21.964052287581698)
(('H', '>250'), 524, 308, 326.7118320610687, 348.97727272727275)
(('S', '100-149'), 777, 50, 103.05855855855856, 100.8)
(('C', '>250'), 152, 46, 304.6510460526315, 320.76052173913047)
(('W', '0-49'), 100046, 1412, 35.98929642364037, 38.15651558073655)
(('R', '50-99'), 4592, 44, 58.54202961672474, 56.81818181818182)
(('R', '200-249'), 4373, 298, 200.79464898239195, 202.68456375838926)
(('R', '100-149'), 11870, 188, 103.24557708508846, 103.8563829787234)
(('C', '150-199'), 826, 117, 170.2330314769975, 167.60790598290598)
(('C', '0-49'), 38117, 4446, 24.770068683264686, 25.886596941070643)
(('H', '100-149'), 5377, 238, 101.10805281755626, 100.97058823529412)
(('C', '100-149'), 2374, 428, 121.20010825610802, 122.84344392523357)
(('R', '>250'), 5741, 502, 391.15572199965163, 365.78685258964146)
(('R', '150-199'), 5235, 174, 151.66666666666666, 156.89655172413794)
(('S', '2

# **Análisis de Dispositivo**

## **¿Hay alguna plataforma más utilizada que otras para cometer fraude?**

Hago un map para hacer un leftOuterJoin entre el dataset de transacciones y el dataset de dispositivos

In [80]:
rdd_transacciones_key = rdd_transacciones.map(lambda row: (row['Id'], row))
rdd_dispositivo_key = rdd_dispositivo.map(lambda row: (row['Id'], row))

rdd_tran_dis = rdd_transacciones_key.leftOuterJoin(rdd_dispositivo_key)

el formato es (ID,(row de transacciones),(row de dataset de dispositivos))

In [81]:
rdd_tran_dis.first()

(2987005,
 (Row(Id=2987005, target=0, timestamp=86510, ammount=49.0, product_category='W', feat_card_1=5937, feat_card_2=555.0, feat_card_3=150.0, feat_card_4='visa', feat_card_5=226.0, feat_card_6='debit', addr1=272.0, addr2=87.0, dist1=36.0, dist2=None, payer_email_domain='gmail.com', receiver_email_domain=None),
  None))

In [82]:
rdd_tran_dis_fraud = rdd_tran_dis.filter(lambda x: x[1][0].target == 1)
rdd_tran_dis_fraud.first()

(2987243,
 (Row(Id=2987243, target=1, timestamp=90246, ammount=37.098, product_category='C', feat_card_1=13413, feat_card_2=103.0, feat_card_3=185.0, feat_card_4='visa', feat_card_5=137.0, feat_card_6='credit', addr1=None, addr2=None, dist1=None, dist2=None, payer_email_domain='hotmail.com', receiver_email_domain='hotmail.com'),
  Row(Id=2987243, feature30=None, feature31='chrome 54.0 for android', Device='mobile', DeviceMetadata='Redmi Note 4 Build/MMB29M')))

In [83]:
rdd_tran_dis_fraud = rdd_tran_dis.filter(lambda x: x[1][0].target == 1)

rdd_tran_dis_fraud.map(lambda x: (x[1][1].Device, 1)) \
    .reduceByKey(lambda x, y: x + y) \
    .map(lambda x: (x[0], x[1])) \
    .collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 110.0 failed 1 times, most recent failure: Lost task 0.0 in stage 110.0 (TID 493) (04d5f382ea6b executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1237, in process
    out_iter = func(split_index, iterator)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/rdd.py", line 5434, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/lib/python3.10/dist-packages/pyspark/rdd.py", line 5434, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/lib/python3.10

Hago un filter solo para quedarme con aquellos registros en los que hay coincidencia entre los dos dataset

In [84]:
rdd_tran_dis_fraud = rdd_tran_dis.filter(lambda x: x[1][0]['target'] == 1)

rdd_tran_dis_fraud \
    .filter(lambda x: x[1][1] is not None) \
    .map(lambda x: (x[1][1]['Device'], 1)) \
    .reduceByKey(lambda x, y: x + y) \
    .map(lambda x: (x[0], x[1])) \
    .collect()

[(None, 71), ('mobile', 4726), ('desktop', 4758)]

In [85]:
rdd_tran_dis_fraud \
    .filter(lambda x: x[1][1] is not None) \
    .map(lambda x: (x[1][1]['Device'], 1)) \
    .reduceByKey(lambda x, y: x + y) \
    .map(lambda x: (x[1])) \
    .sum()

9555

Aqui hago un map para tener todos los registros de los dos dataset

In [86]:
rdd_tran_dis_fraud \
    .map(lambda x: (x[1][1]['Device'] if x[1][1] is not None else 'S/D', 1)) \
    .reduceByKey(lambda x, y: x + y) \
    .map(lambda x: (x[0], x[1])) \
    .collect()

[(None, 71), ('mobile', 4726), ('S/D', 8025), ('desktop', 4758)]

In [87]:
def disp(d):
  if d=='mobile':
    return 'mobile'
  elif d=='desktop':
    return 'desktop'
  else:
    return 'N/A'

Ahora tengo la cantidad de dispositivos en lso que se realizaron operaciones fraudulentas. se separa S/D para aquellos registros que no estan en el dataset de dispositivo de N/A en los que no hay datos en el dataset de dispotitivos

In [88]:
rdd_tran_dis_fraud \
    .map(lambda x: (disp(x[1][1]['Device']) if x[1][1] is not None else 'S/D', 1)) \
    .reduceByKey(lambda x, y: x + y) \
    .map(lambda x: (x[0], x[1])) \
    .collect()

[('mobile', 4726), ('S/D', 8025), ('N/A', 71), ('desktop', 4758)]

## **¿Hay producto y plataforma más utilizada que otras para cometer fraude?**

In [89]:
rdd_tran_dis_fraud.map(lambda x: (
    (x[1][0]['product_category'],
     disp(x[1][1]['Device']) if x[1][1] is not None else 'S/D'),
    (1, x[1][0]['target']))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
    .map(lambda x: (x[0], x[1][1])) \
    .collect()

[(('C', 'S/D'), 342),
 (('R', 'mobile'), 719),
 (('H', 'N/A'), 8),
 (('R', 'N/A'), 14),
 (('R', 'S/D'), 3),
 (('C', 'mobile'), 3133),
 (('H', 'mobile'), 657),
 (('C', 'desktop'), 3220),
 (('S', 'S/D'), 1),
 (('C', 'N/A'), 43),
 (('R', 'desktop'), 478),
 (('H', 'S/D'), 3),
 (('W', 'S/D'), 7676),
 (('S', 'mobile'), 217),
 (('H', 'desktop'), 742),
 (('S', 'N/A'), 6),
 (('S', 'desktop'), 318)]

In [90]:
rdd_tran_dis_fraud.map(lambda x: (
    (x[1][0]['product_category'],
     x[1][1]['Device'] if x[1][1] is not None else 'S/D'),
    (1, x[1][0]['target']))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
    .map(lambda x: (x[1][1])) \
    .sum()

17580

In [91]:
rdd_tran_dis.map(lambda x: (
    (x[1][0]['product_category'],
     disp(x[1][1]['Device']) if x[1][1] is not None else 'S/D'),
    (1, x[1][0]['target']))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
    .map(lambda x: (x[0], x[1][0] ,x[1][1])) \
    .collect()

[(('C', 'S/D'), 5904, 342),
 (('R', 'mobile'), 9924, 719),
 (('H', 'N/A'), 734, 8),
 (('R', 'N/A'), 985, 14),
 (('R', 'S/D'), 140, 3),
 (('C', 'desktop'), 27947, 3220),
 (('H', 'mobile'), 11990, 657),
 (('C', 'mobile'), 24295, 3133),
 (('S', 'S/D'), 36, 1),
 (('C', 'N/A'), 985, 43),
 (('R', 'desktop'), 22460, 478),
 (('H', 'S/D'), 112, 3),
 (('W', 'S/D'), 370202, 7676),
 (('H', 'desktop'), 17626, 742),
 (('S', 'mobile'), 1715, 217),
 (('S', 'N/A'), 233, 6),
 (('S', 'desktop'), 6671, 318)]

In [92]:
rdd_tran_dis.map(lambda x: (
    (x[1][0]['product_category'],
     disp(x[1][1]['Device']) if x[1][1] is not None else 'S/D'),
    (1, x[1][0]['target']))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
    .map(lambda x: (x[1][0])) \
    .sum()

501959

Ahora proporciones absolutas de Legitimas y Fraudulentas y las proporciones relativas de frauduletnas

In [93]:
rdd_tran_dis.map(lambda x: (
    (x[1][0]['product_category'],
     disp(x[1][1]['Device']) if x[1][1] is not None else 'S/D'),
    (1, x[1][0]['target']))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
    .map(lambda x: (x[0], 100* (x[1][0]-x[1][1])/x[1][0] ,x[1][1]/x[1][0],100* x[1][1]/tot_fraud)) \
    .takeOrdered(10, lambda x: -x[3])

[(('W', 'S/D'), 97.92653740390381, 0.020734625960961855, 43.66325369738339),
 (('C', 'desktop'), 88.47819086127312, 0.11521809138726875, 18.31626848691695),
 (('C', 'mobile'), 87.10434245729574, 0.1289565754270426, 17.821387940841866),
 (('H', 'desktop'), 95.79030976965846, 0.04209690230341541, 4.22070534698521),
 (('R', 'mobile'), 92.75493752519145, 0.07245062474808545, 4.089874857792947),
 (('H', 'mobile'), 94.52043369474562, 0.05479566305254378, 3.737201365187713),
 (('R', 'desktop'), 97.87177203918077, 0.02128227960819234, 2.718998862343572),
 (('C', 'S/D'), 94.20731707317073, 0.057926829268292686, 1.9453924914675769),
 (('S', 'desktop'),
  95.23309848598412,
  0.047669015140158894,
  1.8088737201365188),
 (('S', 'mobile'), 87.34693877551021, 0.12653061224489795, 1.2343572241183163)]

## **¿Hay browser y plataforma más utilizada que otras para cometer fraude?**

In [94]:
def browser(b):
  if b==None:
    return 'N/A'
  else:
    return b

In [95]:
rdd_tran_dis.map(lambda x: (
    (browser(x[1][1]['feature30']) if x[1][1] is not None else 'S/D',
     disp(x[1][1]['Device']) if x[1][1] is not None else 'S/D'),
    (1, x[1][0]['target']))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
    .map(lambda x: (x[0], 100* (x[1][0]-x[1][1])/x[1][0] ,x[1][1]/x[1][0],100* x[1][1]/tot_fraud)) \
    .takeOrdered(10, lambda x: -x[3])

[(('S/D', 'S/D'), 97.86792563112058, 0.021320743688794188, 45.648464163822524),
 (('N/A', 'desktop'),
  88.75440737727149,
  0.11245592622728505,
  18.86803185437998),
 (('N/A', 'mobile'),
  87.11723801390752,
  0.12882761986092472,
  18.02047781569966),
 (('Windows 10', 'desktop'),
  96.85910691331806,
  0.03140893086681942,
  3.196814562002275),
 (('Windows 7', 'desktop'),
  96.65071770334929,
  0.03349282296650718,
  2.2696245733788394),
 (('iOS 11.2.1', 'mobile'),
  95.03257328990227,
  0.0496742671009772,
  1.0409556313993173),
 (('Android 7.0', 'mobile'),
  94.6236559139785,
  0.053763440860215055,
  0.8248009101251422),
 (('iOS 11.2.5', 'mobile'),
  87.78761061946902,
  0.12212389380530973,
  0.7849829351535836),
 (('iOS 11.1.2', 'mobile'),
  96.46547036432844,
  0.03534529635671561,
  0.7394766780432309),
 (('Windows 8.1', 'desktop'),
  92.6205694363742,
  0.073794305636258,
  0.7224118316268487)]

Aqui me fijo si hay alguna combinacion de OS y Browser con mas operaciones fraudulentas

In [96]:
brow_os=rdd_tran_dis.map(lambda x: (
    (browser(x[1][1]['feature30']) if x[1][1] is not None else 'S/D',
     disp(x[1][1]['Device']) if x[1][1] is not None else 'S/D'),
    (1, x[1][0]['target']))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
    .map(lambda x: (x[0], 100* (x[1][0]-x[1][1])/x[1][0] ,x[1][1]/x[1][0],100* x[1][1]/tot_fraud)) \
    .takeOrdered(10, lambda x: -x[2])

In [97]:
for os in brow_os:
  print(os)

(('other', 'desktop'), 25.0, 0.75, 0.017064846416382253)
(('Android 5.1.1', 'mobile'), 69.1358024691358, 0.30864197530864196, 0.5688282138794084)
(('iOS 11.0.1', 'mobile'), 78.99159663865547, 0.21008403361344538, 0.1422070534698521)
(('Linux', 'mobile'), 80.0, 0.2, 0.022753128555176336)
(('iOS 11.1.0', 'mobile'), 81.60919540229885, 0.1839080459770115, 0.27303754266211605)
(('Android 4.4.2', 'mobile'), 84.90566037735849, 0.1509433962264151, 0.04550625711035267)
(('Windows 8', 'desktop'), 87.09677419354838, 0.12903225806451613, 0.11376564277588168)
(('N/A', 'mobile'), 87.11723801390752, 0.12882761986092472, 18.02047781569966)
(('Mac OS X 10_12_1', 'desktop'), 87.5, 0.125, 0.02844141069397042)
(('iOS 11.2.5', 'mobile'), 87.78761061946902, 0.12212389380530973, 0.7849829351535836)


Se puede ver que las que tienen mas operaciones fraudulentas